# Analysis

- Create separate notebook as 'poc.ipynb' notebook is too bloated.


In [2]:
import pandas as pd
from abc import ABC, abstractmethod
from datetime import datetime, date
import re
from collections import deque, Counter, defaultdict
from decimal import Decimal
import numpy as np
import yfinance as yf
from pydantic import BaseModel, Field, computed_field, field_validator
import math
from pathlib import Path
from zoneinfo import ZoneInfo
import calendar
import itertools
from omegaconf import OmegaConf
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Any, get_args
from pprint import pformat
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib inline

from src.utils import utils, plot_utils, main_utils, strategy_utils, strategy_utils
from config.variables import EntryType, CointCorrFn, StratComponent
from src.strategy.base.stock_trade import StockTrade
from src.cal_coint_corr import CalCointCorr
from src.plot_strategies import PlotStrategies

cfg = utils.load_config()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
10 // 4 * 4

8

In [6]:
dt = "2025-04-01"
date_dir = f"./data/{dt}"
strat_comp_dict_path = f"{date_dir}/strat_comp_dict.pkl"

strat_comp_dict = utils.load_pickle(strat_comp_dict_path)
strat_comp_dict

defaultdict(dict,
            {'entry_type': {'annualized_return':                   min       max       mean     median
              entry_type                                           
              long        -0.332153  0.218453  -0.029100  -0.023017
              longshort   -0.109013  0.112047   0.022765   0.028508
              short       -0.073799  0.471104   0.089277   0.082365,
              'win_rate':                  min       max      mean    median
              entry_type                                        
              long        0.319347  0.619847  0.506809  0.506621
              longshort   0.435819  0.597198  0.526276  0.534146
              short       0.381423  0.674208  0.535541  0.539626,
              'max_days_held':                   min        max       mean     median
              entry_type                                            
              long        19.000000  27.000000  25.672743  27.000000
              longshort   19.000000  27.0000

In [13]:
cfg.plot_strat.drilldown_mapping

{'entry_type': 'Entry Strategy', 'entry_struct': 'Entry Method', 'exit_struct': 'Exit Method', 'stop_method': 'Stop Loss Method', 'hf_model': 'FinBERT Model', 'coint_corr_fn': 'Cointegration / Correlation Function', 'period': 'Time Period'}

# Table of Contents

1. [Cointegration / Correlation](#cointegration--correlation)
2. [Overall](#overall)
   - 2.1 [Overall Combined](#overall-combined)
   - 2.2 [Top 15 Tickers](#top-15-tickers)
   - 2.3 [Pivot Top 15 Tickers](#pivot-top-15-tickers)
3. [Specific Strategy](#specific-strategy)
   - 3.1 [Trade Results](#trade-results)
   - 3.2 [Overall Summary](#overall-summary)
   - 3.3 [Breakdown Summary](#breakdown-summary)
   - 3.4 [Top Ticker Pairs](#top-ticker-pairs)
4. [Test Trade Generation](#test-trade-generation)
5. [GenTrades](#gentrades)


# [Cointegration / Correlation](#table-of-contents)


In [4]:
snp500_list = utils.gen_snp500_list(cfg.snp500_url, cfg.ignore_list)

cal_cc = CalCointCorr(
    path=cfg.path, snp500_list=snp500_list, date="2025-04-01", **cfg.coint_corr
)
ticker_dict = cal_cc.get_kendall_tickers()
ticker_dict

{1: ['A',
  'AIG',
  'BDX',
  'TECH',
  'BA',
  'CSGP',
  'CVS',
  'EOG',
  'GEV',
  'HSIC',
  'INTU',
  'MSFT',
  'PARA',
  'ROST',
  'SW',
  'SOLV',
  'STLD',
  'SYY',
  'TEL',
  'TSCO',
  'YUM'],
 3: ['BDX',
  'CHTR',
  'CVX',
  'STZ',
  'EW',
  'F',
  'GEHC',
  'GEV',
  'GPN',
  'GL',
  'IEX',
  'INTC',
  'JNJ',
  'KVUE',
  'LULU',
  'MKTX',
  'MTD',
  'MOH',
  'NOC',
  'PEP',
  'PLD',
  'SW',
  'SOLV',
  'TSLA',
  'UNH',
  'VLTO'],
 5: ['APD',
  'ABNB',
  'AKAM',
  'T',
  'BDX',
  'BIIB',
  'CPB',
  'CAG',
  'CEG',
  'CSGP',
  'DXCM',
  'DASH',
  'EXE',
  'GEHC',
  'GEV',
  'HOLX',
  'PODD',
  'JKHY',
  'KVUE',
  'KMB',
  'LULU',
  'PLTR',
  'SW',
  'SOLV',
  'SBUX',
  'VLTO',
  'VRSN']}

In [5]:
cal_cc.get_kendall_stats(ticker_dict)

Intersection between period 1 and period 3 [total: 4]: {'SW', 'BDX', 'GEV', 'SOLV'}
Intersection between period 1 and period 5 [total: 5]: {'SW', 'GEV', 'CSGP', 'SOLV', 'BDX'}
Intersection between period 3 and period 5 [total: 8]: {'SW', 'GEV', 'GEHC', 'SOLV', 'VLTO', 'BDX', 'LULU', 'KVUE'}

Common tickers : 

{'SW', 'BDX', 'GEV', 'SOLV'}

All tickers : 

{'INTC', 'BA', 'INTU', 'AIG', 'MSFT', 'YUM', 'NOC', 'EW', 'GEHC', 'A', 'TSLA', 'CAG', 'CPB', 'HOLX', 'EOG', 'F', 'SBUX', 'VRSN', 'LULU', 'KVUE', 'GL', 'SW', 'GPN', 'APD', 'ROST', 'GEV', 'MTD', 'SOLV', 'CEG', 'VLTO', 'T', 'TECH', 'DXCM', 'STZ', 'UNH', 'HSIC', 'PODD', 'EXE', 'CVS', 'BIIB', 'TSCO', 'BDX', 'JNJ', 'CHTR', 'PEP', 'PLD', 'KMB', 'IEX', 'JKHY', 'MOH', 'PARA', 'PLTR', 'SYY', 'CSGP', 'CVX', 'STLD', 'TEL', 'ABNB', 'DASH', 'MKTX', 'AKAM'}



# [Overall](#table-of-contents)


## [Overall Combined](#table-of-contents)


In [3]:
dt = "2025-04-01"
combined = utils.load_csv(f"./data/{dt}/combined_overall.csv", tz="America/New_York")
combined

,entry_type,entry_struct,exit_struct,stop_method,hf_model,coint_corr_fn,period,total_num_stock_tickers,num_tickers_with_no_trades,stock_ticker_without_trades,...,last_trade,min_days_held,max_days_held,mean_days_held,median_days_held,trading_period,total_investment,total_profit,percent_return,annualized_return
0,long,multiple,fifo,no_stop,prosusai,coint,1,19,2,"NVDA, TSLA",...,2025-03-28 16:00:00-04:00,1.0,26.0,8.422487223168654,7.0,27,194097.67,129.52,0.000667,0.009055
1,long,multiple,fifo,no_stop,prosusai,coint,3,19,2,"NVDA, TSLA",...,2025-03-28 16:00:00-04:00,1.0,26.0,8.392491467576791,7.0,27,222155.04,714.3,0.003215,0.044348
2,long,multiple,fifo,no_stop,prosusai,coint,5,19,2,"NVDA, TSLA",...,2025-03-28 16:00:00-04:00,1.0,26.0,8.389830508474576,7.0,27,222340.73,-16.76,-0.000075,-0.001013
3,long,multiple,fifo,no_stop,prosusai,pearsonr,1,19,2,"NVDA, TSLA",...,2025-03-28 16:00:00-04:00,1.0,26.0,8.37231298366294,7.0,27,284912.52,-101.04,-0.000355,-0.004788
4,long,multiple,fifo,no_stop,prosusai,pearsonr,3,19,2,"NVDA, TSLA",...,2025-03-28 16:00:00-04:00,1.0,26.0,8.389830508474576,7.0,27,402182.55,-1832.48,-0.004556,-0.059864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5179,longshort,single,take_all,latest_loss,aventiq_ai,spearmanr,3,19,0,NaN,...,2025-03-28 16:00:00-04:00,1.0,24.0,3.877319587628866,3.0,27,293333.24,1451.34,0.004948,0.069001
5180,longshort,single,take_all,latest_loss,aventiq_ai,spearmanr,5,19,0,NaN,...,2025-03-28 16:00:00-04:00,1.0,24.0,3.9342379958246343,3.0,27,304441.55,1680.08,0.005519,0.077241
5181,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,1,19,2,"MSFT, BA",...,2025-03-28 16:00:00-04:00,1.0,24.0,3.864864864864865,3.0,27,162599.15,685.71,0.004217,0.058537
5182,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,3,19,3,"TSLA, JNJ, UNH",...,2025-03-28 16:00:00-04:00,1.0,24.0,3.753390875462392,3.0,27,278314.67,1138.93,0.004092,0.056757


In [58]:
a = combined.copy()
a = a.groupby("entry_type").agg({"annualized_return": ["min", "max", "mean", "median"]})
# a.columns = ["_".join(col_tuple) for col_tuple in a.columns]
a.columns = [cols[1] for cols in a.columns]
a

,min,max,mean,median
entry_type,,,,
long,-0.332153,0.218453,-0.0291,-0.023017
longshort,-0.109013,0.112047,0.022765,0.028508
short,-0.073799,0.471104,0.089277,0.082365


In [15]:
a = combined.loc[
    :, ["annualized_return", "max_days_held", "win_rate", "highest_neg_percent_return"]
].copy()
a

,annualized_return,max_days_held,win_rate,highest_neg_percent_return
0,0.009055,26.0,0.528109,-0.187663
1,0.044348,26.0,0.488908,-0.186463
2,-0.001013,26.0,0.605085,-0.141297
3,-0.004788,26.0,0.549441,-0.187663
4,-0.059864,26.0,0.499153,-0.123324
...,...,...,...,...
5179,0.069001,24.0,0.54433,-0.063159
5180,0.077241,24.0,0.553236,-0.06397
5181,0.058537,24.0,0.539312,-0.063159
5182,0.056757,24.0,0.545006,-0.063159


In [18]:
a = a.astype(float)
a.describe().T

,count,mean,std,min,25%,50%,75%,max
annualized_return,5184.0,0.029275,0.090038,-0.332153,-0.025554,0.026671,0.080592,0.471104
max_days_held,5184.0,22.317323,5.419522,9.000000,19.000000,25.000000,27.000000,27.000000
win_rate,5184.0,0.521742,0.050562,0.319347,0.489108,0.526708,0.552548,0.674208
highest_neg_percent_return,5184.0,-0.102771,0.044263,-0.310415,-0.135512,-0.093755,-0.063159,-0.050006


In [4]:
combined_path = f"./data/{dt}/combined_overall.csv"
main_utils.gen_stats_df(combined_path)

,count,mean,std,min,25%,50%,75%,max
annualized_return,5184.000000,0.029275,0.090038,-0.332153,-0.025554,0.026671,0.080592,0.471104
win_rate,5184.000000,0.521742,0.050562,0.319347,0.489108,0.526708,0.552548,0.674208
max_days_held,5184.000000,22.317323,5.419522,9.000000,19.000000,25.000000,27.000000,27.000000
highest_neg_percent_return,5184.000000,-0.102771,0.044263,-0.310415,-0.135512,-0.093755,-0.063159,-0.050006


In [35]:
df_dict = plot_utils.gen_stats_comp_stats(combined_path)
df_dict

defaultdict(dict,
            {'entry_type': {'annualized_return':                   min       max       mean     median
              entry_type                                           
              long        -0.332153  0.218453  -0.029100  -0.023017
              longshort   -0.109013  0.112047   0.022765   0.028508
              short       -0.073799  0.471104   0.089277   0.082365,
              'win_rate':                  min       max      mean    median
              entry_type                                        
              long        0.319347  0.619847  0.506809  0.506621
              longshort   0.435819  0.597198  0.526276  0.534146
              short       0.381423  0.674208  0.535541  0.539626,
              'max_days_held':                   min        max       mean     median
              entry_type                                            
              long        19.000000  27.000000  25.672743  27.000000
              longshort   19.000000  27.0000

In [40]:
cfg.plot_strat.analysis_cols

['annualized_return', 'win_rate', 'max_days_held', 'highest_neg_percent_return']

In [42]:
strat_comp = "entry_type"
attribute = "max_days_held"
df_dict[strat_comp][attribute]

,min,max,mean,median
entry_type,,,,
long,19.000000,27.000000,25.672743,27.000000
longshort,19.000000,27.000000,23.852431,24.000000
short,9.000000,25.000000,18.578125,20.000000


## [Top 15 Tickers](#table-of-contents)


In [77]:
df_top_n = utils.load_csv("./data/2025-04-01/top_15_tickers.csv")
df_top_n

,strategy,entry_type,entry_struct,exit_struct,stop_method,hf_model,coint_corr_fn,period,ticker_pair,news_ticker,ticker,overall_daily_ret,days_held_max,trading_period,win_rate,win_count,neg_ret_mean,neg_ret_max
0,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,GOOGL_MOH,GOOGL,MOH,0.012535,2.0,3,1.0,2,NaN,NaN
1,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,GOOGL_SJM,GOOGL,SJM,0.011973,2.0,3,1.0,2,NaN,NaN
2,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,GOOGL_MDLZ,GOOGL,MDLZ,0.009475,2.0,3,1.0,2,NaN,NaN
3,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,GOOGL_XOM,GOOGL,XOM,0.005379,2.0,3,1.0,2,NaN,NaN
4,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,GOOGL_CI,GOOGL,CI,0.004546,2.0,3,1.0,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77755,longshort_single_take_all_latest_loss_aventiq_...,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,5,MSFT_AAPL,MSFT,AAPL,0.002353,7.0,11,1.0,2,NaN,NaN
77756,longshort_single_take_all_latest_loss_aventiq_...,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,5,MSFT_DELL,MSFT,DELL,0.001897,7.0,11,0.5,2,-0.018399,-0.018399
77757,longshort_single_take_all_latest_loss_aventiq_...,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,5,AAPL_NVDA,AAPL,NVDA,0.001588,7.0,11,0.5,2,-0.027371,-0.027371
77758,longshort_single_take_all_latest_loss_aventiq_...,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,5,AAPL_PANW,AAPL,PANW,0.001453,7.0,11,0.5,2,-0.021784,-0.021784


In [63]:
df_top_3 = main_utils.get_top_pairs_by_entry_type(
    f"./data/2025-04-01/top_15_tickers.csv", top_n=3
)
df_top_3

,category,ticker_pair,overall_daily_ret,days_held_max,trading_period,win_rate,win_count,neg_ret_mean,neg_ret_max
0,long,GOOGL_MOH,0.018838,2.0,2,1.0,2,NaN,NaN
1,long,AMZN_DFS,0.015333,4.0,4,1.0,2,NaN,NaN
2,long,GOOGL_APTV,0.014679,2.0,2,1.0,1,NaN,NaN
3,short,GOOGL_APTV,0.038835,2.0,2,1.0,1,NaN,NaN
4,short,GOOGL_PLTR,0.034253,2.0,2,1.0,1,NaN,NaN
5,short,GOOGL_GOOG,0.032614,2.0,2,1.0,1,NaN,NaN
6,longshort,TSLA_CZR,0.018107,4.0,4,1.0,1,NaN,NaN
7,longshort,NVDA_CZR,0.018107,4.0,4,1.0,1,NaN,NaN
8,longshort,TSLA_UAL,0.014896,4.0,4,1.0,1,NaN,NaN


In [28]:
df_top_n["strategy"].value_counts()

strategy
long_multiple_half_fifo_latest_loss_yiyanghkust_kendalltau_5    30
long_multiple_half_fifo_latest_loss_ziweichen_coint_1           30
long_multiple_half_fifo_latest_loss_ziweichen_coint_3           30
long_multiple_half_fifo_latest_loss_ziweichen_coint_5           30
long_multiple_half_fifo_latest_loss_ziweichen_pearsonr_1        30
                                                                ..
short_multiple_fifo_percent_loss_yiyanghkust_kendalltau_1       15
short_multiple_fifo_percent_loss_yiyanghkust_kendalltau_3       15
short_multiple_fifo_percent_loss_yiyanghkust_kendalltau_5       15
short_multiple_fifo_percent_loss_ziweichen_coint_1              15
short_multiple_fifo_percent_loss_prosusai_kendalltau_3          15
Name: count, Length: 4416, dtype: int64

## [Pivot Top 15 Tickers](#table-of-contents)


In [ ]:
df_pivot_top_n = utils.load_csv("./data/2025-04-01/pivot_top_15_tickers.csv")
df_pivot_top_n

,strategy,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,top_11,top_12,top_13,top_14,top_15
0,long_multiple_fifo_no_stop_prosusai_coint_1,AMZN_CI,GOOGL_CI,GOOGL_MDLZ,GOOGL_MOH,GOOGL_SJM,GOOGL_XOM,HD_MSI,JNJ_CDNS,JNJ_EOG,JPM_CTVA,META_CTVA,MSFT_EOG,NFLX_BR,UNH_ULTA,XOM_SJM
1,long_multiple_fifo_no_stop_prosusai_coint_3,AMZN_BR,AMZN_NRG,AMZN_WM,HD_EOG,JNJ_CTRA,JNJ_CVX,JNJ_NEM,META_ADM,META_CZR,NFLX_AZO,UNH_NOC,V_DVN,WMT_DLTR,WMT_EOG,XOM_HCA
2,long_multiple_fifo_no_stop_prosusai_coint_5,AMZN_DLR,AMZN_DLTR,AMZN_EA,AMZN_ECL,BA_JKHY,HD_CAG,HD_EA,JNJ_KDP,META_CAG,PG_TSCO,UNH_NOC,XOM_CI,XOM_EA,XOM_ED,XOM_NOC
3,long_multiple_fifo_no_stop_prosusai_pearsonr_1,AMZN_COF,AMZN_DFS,AMZN_TSLA,BA_DG,BA_DLTR,BA_DPZ,JNJ_MMC,JPM_BR,JPM_DFS,META_LYV,META_MA,MSFT_ROST,NFLX_FOX,V_DRI,XOM_CI
4,long_multiple_fifo_no_stop_prosusai_pearsonr_3,AMZN_BR,AMZN_COST,AMZN_CRWD,AMZN_NOW,BA_SLB,META_MA,MSFT_CMG,MSFT_CPRT,PG_HIG,UNH_EA,UNH_NOC,XOM_ACGL,XOM_FANG,XOM_HES,XOM_PCG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5179,longshort_single_take_all_latest_loss_aventiq_...,AAPL_AVGO,BA_ON,MSFT_CMG,MSFT_DELL,MSFT_GOOG,MSFT_GOOGL,MSFT_META,MSFT_NOW,NVDA_AVGO,NVDA_BKNG,NVDA_GE,NVDA_META,NVDA_PNR,NVDA_RCL,TSLA_GNRC
5180,longshort_single_take_all_latest_loss_aventiq_...,AAPL_AVGO,BAC_CBRE,JPM_KKR,META_LII,META_MAS,META_PNR,MSFT_AAPL,MSFT_DELL,NVDA_APH,NVDA_APO,NVDA_AVGO,NVDA_ETN,NVDA_MSFT,NVDA_NVR,NVDA_ORCL
5181,longshort_single_take_all_latest_loss_aventiq_...,JPM_DFS,JPM_WMT,META_COST,META_IBM,META_TKO,META_WMT,NVDA_ANET,NVDA_APH,NVDA_HPE,NVDA_LII,NVDA_ORCL,NVDA_TT,TSLA_CCL,TSLA_GLW,V_DRI
5182,longshort_single_take_all_latest_loss_aventiq_...,META_PNR,MSFT_CMG,MSFT_DELL,MSFT_GOOG,MSFT_GOOGL,MSFT_META,MSFT_NOW,NVDA_ANET,NVDA_AVGO,NVDA_BKNG,NVDA_GE,NVDA_META,NVDA_PNR,NVDA_RCL,V_HWM


# [Specific Strategy](#table-of-contents)


## [Trade Results](#table-of-contents)


In [3]:
cfg = utils.load_config()
date = "2025-04-01"
model_dir = f"{cfg.path.data_dir}/{date}/{cfg.single.entry_type}_{cfg.single.entry_struct}_{cfg.single.exit_struct}_{cfg.single.stop_method}/{cfg.single.hf_model}_{cfg.single.coint_corr_fn}_{cfg.single.period}"

results = utils.load_csv(
    f"{model_dir}/trade_results.csv",
    tz="America/New_York",
)
results

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,AAPL,ANET,2025-03-27 16:00:00-04:00,buy,10,78.96,2025-03-28 16:00:00-04:00,sell,10,77.94,1.0,-1.02,-0.012918,-0.012918,0.0
1,AAPL,APH,2025-03-27 16:00:00-04:00,buy,10,68.17,2025-03-28 16:00:00-04:00,sell,10,66.59,1.0,-1.58,-0.023177,-0.023177,0.0
2,AAPL,AVGO,2025-03-27 16:00:00-04:00,buy,10,171.99,2025-03-28 16:00:00-04:00,sell,10,169.12,1.0,-2.87,-0.016687,-0.016687,0.0
3,AAPL,BRO,2025-03-27 16:00:00-04:00,buy,10,122.9,2025-03-28 16:00:00-04:00,sell,10,122.93,1.0,0.03,0.000244,0.000244,1.0
4,AAPL,COST,2025-03-27 16:00:00-04:00,buy,10,938.75,2025-03-28 16:00:00-04:00,sell,10,929.66,1.0,-9.09,-0.009683,-0.009683,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,BA,WYNN,2025-03-19 16:00:00-04:00,buy,10,85.05,2025-03-25 16:00:00-04:00,sell,10,86.03,6.0,0.98,0.011523,0.001911,1.0
1055,BA,WYNN,2025-03-20 16:00:00-04:00,buy,10,83.07,2025-03-25 16:00:00-04:00,sell,10,86.03,5.0,2.96,0.035633,0.007027,1.0
1056,BA,WYNN,2025-03-21 16:00:00-04:00,buy,10,82.63,2025-03-25 16:00:00-04:00,sell,10,86.03,4.0,3.4,0.041147,0.010132,1.0
1057,BA,WYNN,2025-03-24 16:00:00-04:00,buy,10,84.87,2025-03-25 16:00:00-04:00,sell,10,86.03,1.0,1.16,0.013668,0.013668,1.0


## [Overall Summary](#table-of-contents)


In [24]:
overall = utils.load_csv(f"{model_dir}/overall_summary.csv", index_col=0)
overall

,Overall Statistics
entry_type,long
entry_struct,multiple
exit_struct,take_all
stop_method,no_stop
hf_model,prosusai
coint_corr_fn,kendalltau
period,5
total_num_stock_tickers,19
num_tickers_with_no_trades,2
stock_ticker_without_trades,"NVDA, TSLA"


## [Breakdown Summary](#table-of-contents)


In [29]:
breakdown = utils.load_csv(
    f"{model_dir}/breakdown_summary.csv",
    tz="America/New_York",
)
breakdown

,news_ticker,ticker,entry_datetime_min,entry_datetime_max,exit_datetime_max,days_held_min,days_held_max,days_held_mean,days_held_median,entry_price_sum,...,win_count,win_sum,win_rate,trading_period,overall_percent_ret,overall_daily_ret,neg_ret_max,neg_ret_min,neg_ret_mean,neg_ret_median
0,AAPL,ANET,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,78.96,...,1,0.0,0.0,1,-0.012918,-0.012918,-0.012918,-0.012918,-0.012918,-0.012918
1,AAPL,APH,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,68.17,...,1,0.0,0.0,1,-0.023177,-0.023177,-0.023177,-0.023177,-0.023177,-0.023177
2,AAPL,AVGO,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,171.99,...,1,0.0,0.0,1,-0.016687,-0.016687,-0.016687,-0.016687,-0.016687,-0.016687
3,AAPL,BRO,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,122.9,...,1,1.0,1.0,1,0.000244,0.000244,NaN,NaN,NaN,NaN
4,AAPL,COST,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,938.75,...,1,0.0,0.0,1,-0.009683,-0.009683,-0.009683,-0.009683,-0.009683,-0.009683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,XOM,MPC,2025-03-03 16:00:00-05:00,2025-03-28 16:00:00-04:00,2025-03-31 16:00:00-04:00,1.0,12.0,5.083333333333333,4.5,1711.04,...,12,6.0,0.5,27,0.008591,0.000317,-0.056714,-0.000073,-0.026918,-0.026721
154,XOM,PSX,2025-03-03 16:00:00-05:00,2025-03-28 16:00:00-04:00,2025-03-31 16:00:00-04:00,1.0,12.0,5.083333333333333,4.5,1492.59,...,12,8.0,0.666667,27,0.013473,0.000496,-0.042791,-0.016017,-0.02481,-0.020215
155,XOM,TKO,2025-03-03 16:00:00-05:00,2025-03-28 16:00:00-04:00,2025-03-31 16:00:00-04:00,1.0,12.0,5.083333333333333,4.5,1776.23,...,12,6.0,0.5,27,0.000203,0.000008,-0.056291,-0.008443,-0.022934,-0.016906
156,XOM,TRGP,2025-03-03 16:00:00-05:00,2025-03-28 16:00:00-04:00,2025-03-31 16:00:00-04:00,1.0,12.0,5.083333333333333,4.5,2307.65,...,12,7.0,0.583333,27,0.011627,0.000428,-0.07037,-0.000385,-0.025913,-0.024809


In [17]:
breakdown.columns

Index(['news_ticker', 'ticker', 'entry_datetime_min', 'entry_datetime_max',
       'exit_datetime_max', 'days_held_min', 'days_held_max', 'days_held_mean',
       'days_held_median', 'entry_price_sum', 'profit_loss_sum',
       'percent_ret_min', 'percent_ret_max', 'percent_ret_mean',
       'percent_ret_median', 'win_count', 'win_sum', 'trading_period',
       'overall_percent_ret', 'overall_daily_ret', 'neg_ret_max',
       'neg_ret_min', 'neg_ret_mean', 'neg_ret_median'],
      dtype='object')

## [Top Ticker Pairs](#table-of-contents)


In [118]:
top_n = utils.load_csv(
    f"{model_dir}/top_ticker_pairs.csv",
    tz="America/New_York",
)
top_n

,news_ticker,ticker,entry_datetime_min,entry_datetime_max,exit_datetime_max,days_held_min,days_held_max,days_held_mean,days_held_median,entry_price_sum,...,daily_ret_median,win_count,win_sum,trading_period,overall_percent_ret,overall_daily_ret,neg_ret_max,neg_ret_min,neg_ret_mean,neg_ret_median
0,NVDA,APH,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-04-01 16:00:00-04:00,5,5,5.0,5.0,68.17,...,0.006086,1,1,5,0.030805,0.006086,NaN,NaN,NaN,NaN
1,MSFT,CMG,2025-03-20 16:00:00-04:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,7,4.0,4.0,150.13,...,0.005506,3,3,12,0.027243,0.002242,NaN,NaN,NaN,NaN
2,AAPL,AVGO,2025-03-20 16:00:00-04:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,7,4.0,4.0,529.96,...,-0.00651,3,1,12,0.024341,0.002006,-0.026513,-0.00651,-0.016511,-0.016511
3,META,LII,2025-03-18 16:00:00-04:00,2025-03-21 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,11,4.666667,2.0,1742.35,...,0.017304,3,2,14,0.015623,0.001108,-0.008287,-0.008287,-0.008287,-0.008287
4,BAC,CBRE,2025-03-03 16:00:00-05:00,2025-03-26 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,7,4.0,5.0,926.24,...,0.005956,7,5,28,0.031385,0.001104,-0.019159,-0.01214,-0.01565,-0.01565
5,JPM,KKR,2025-03-03 16:00:00-05:00,2025-03-27 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,11,3.5,1.5,959.74,...,0.0054565,8,5,28,0.028706,0.001011,-0.0385,-0.001722,-0.020883,-0.022426
6,JNJ,KDP,2025-03-03 16:00:00-05:00,2025-03-26 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,15,7.0,6.0,134.56,...,0.0034995,4,3,28,0.026457,0.000933,-0.006799,-0.006799,-0.006799,-0.006799
7,V,GE,2025-03-03 16:00:00-05:00,2025-03-06 16:00:00-05:00,2025-04-01 16:00:00-04:00,1,25,9.333333,2.0,599.39,...,0.001624,3,3,28,0.025142,0.000887,NaN,NaN,NaN,NaN
8,UNH,NOC,2025-03-03 16:00:00-05:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,15,4.0,2.0,3419.19,...,0.002129,7,6,28,0.023985,0.000847,-0.002109,-0.002109,-0.002109,-0.002109
9,BA,PPG,2025-03-10 16:00:00-04:00,2025-03-28 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,6,3.666667,3.5,671.52,...,0.003995,6,5,22,0.016708,0.000753,-0.012025,-0.012025,-0.012025,-0.012025


# [Test Trade Generation](#table-of-contents)


In [14]:
trade = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-01",
    entry_action="buy",
    entry_lots=10,
    entry_price=200,
)
trade1 = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-02",
    entry_action="buy",
    entry_lots=10,
    entry_price=195,
)

open_trades = deque([trade, trade1])
entry_method = strategy_utils.get_class_instance(
    "MultiEntry", "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-4-4", "buy", 192)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [19]:
utils.display_open_trades(open_trades)

open_trades : 
[
   ticker: 'AAPL', ent_dt: '2025-04-01', ent_act: 'buy', ent_lots: 10, ent_price: 200, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
   ticker: 'AAPL', ent_dt: '2025-04-02', ent_act: 'buy', ent_lots: 10, ent_price: 195, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
   ticker: 'AAPL', ent_dt: '2025-04-04', ent_act: 'buy', ent_lots: 20, ent_price: 192, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
]



In [140]:
open_trades = deque()
entry_method = strategy_utils.get_class_instance(
    "MultiHalfEntry", "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-01", "sell", 192)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-02", "sell", 196)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-03", "sell", 193)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='sell', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='sell', entry_lots=Decimal('10'), entry_price=Decimal('196'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='sell', entry_lots=Decimal('5'), entry_price=Decimal('193'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [50]:
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [51]:
exit_method = strategy_utils.get_class_instance(
    "HalfLIFOExit", "./src/strategy/base/exit_struct.py"
)
open_trades, completed_trades = exit_method.close_pos(
    open_trades, "2025-04-10", "buy", 180
)
open_trades

Validation Error : 1 validation error for StockTrade
exit_action
  Value error, Entry action cannot be the same as exit action except when both are 'wait'. [type=value_error, input_value='buy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [131]:
open_trades = deque()
entry_method = strategy_utils.get_class_instance(
    "MultiHalfEntry", "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-01", "buy", 192)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-02", "buy", 196)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-03", "buy", 193)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('196'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='buy', entry_lots=Decimal('5'), entry_price=Decimal('193'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [141]:
stop_price = strategy_utils.get_class_instance(
    "PercentLoss", "./src/strategy/base/cal_exit_price.py", percent_loss=0.1
)
stop_price.cal_exit_price(open_trades)

Decimal('212.61')

In [30]:
pos = "long"
trade1 = (10, 141.87)
trade2 = (10, 141.97)
trade3 = (10, 143.25)
percent_loss = 0.001
trades = [
    trade1,
    trade2,
    trade3,
]
total_lots = sum(trade[0] for trade in trades)
total_invest = sum(trade[0] * trade[1] for trade in trades)
# stop_loss = [trade[1] * (1-percent_loss) if pos == "long" else trade[1] * (1+percent_loss) for trade in trades]
# stop_loss = [Decimal(str(round(loss, 2))) for loss in stop_loss]
# stop_loss

# (10 * 200 + 10 * 195 + 20 * 192 ) * (1-percent_loss)/ 40

stop_price = (
    total_invest * (1 - percent_loss) / total_lots
    if pos == "long"
    else total_invest * (1 + percent_loss) / total_lots
)
stop_price

142.22096999999997

In [32]:
142.9 * (1 - percent_loss)

142.7571

# [GenTrades](#table-of-contents)


In [58]:
date = "2025-04-01"
entry_type = "longshort"
entry_struct = "multiple"
exit_struct = "take_all"
stop_method = "no_stop"
model = "ziweichen_coint_5"

date_dir = f"./data/{date}"
results_dir = f"{date_dir}/{entry_type}_{entry_struct}_{exit_struct}_{stop_method}"
model_dir = f"{results_dir}/{model}"
pa_dir = f"{model_dir}/price_actions"

In [67]:
corr = pd.read_csv("./data/coint_corr/2025-04-01/coint_corr_1y.csv")
corr

,ticker1,ticker2,coint,pearsonr,spearmanr,kendalltau
0,MMM,AOS,0.318636,-0.739837,-0.674401,-0.504392
1,MMM,ABT,0.037253,0.844638,0.841827,0.605854
2,MMM,ABBV,0.247177,0.795093,0.763711,0.565531
3,MMM,ACN,0.701204,0.713788,0.675311,0.486818
4,MMM,ADBE,0.603408,-0.332484,-0.339570,-0.217770
...,...,...,...,...,...,...
123251,YUM,ZBH,0.958000,-0.032467,0.140320,0.094599
123252,YUM,ZTS,0.864792,-0.295493,-0.281127,-0.178846
123253,ZBRA,ZBH,0.481884,-0.410229,-0.326750,-0.203995
123254,ZBRA,ZTS,0.847695,0.377118,0.407230,0.289736


In [70]:
pd.concat([corr, pd.DataFrame()], axis=0)

,ticker1,ticker2,coint,pearsonr,spearmanr,kendalltau
0,MMM,AOS,0.318636,-0.739837,-0.674401,-0.504392
1,MMM,ABT,0.037253,0.844638,0.841827,0.605854
2,MMM,ABBV,0.247177,0.795093,0.763711,0.565531
3,MMM,ACN,0.701204,0.713788,0.675311,0.486818
4,MMM,ADBE,0.603408,-0.332484,-0.339570,-0.217770
...,...,...,...,...,...,...
123251,YUM,ZBH,0.958000,-0.032467,0.140320,0.094599
123252,YUM,ZTS,0.864792,-0.295493,-0.281127,-0.178846
123253,ZBRA,ZBH,0.481884,-0.410229,-0.326750,-0.203995
123254,ZBRA,ZTS,0.847695,0.377118,0.407230,0.289736


In [59]:
results = utils.load_csv(f"{model_dir}/trade_results.csv", tz="America/New_York")
results

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,AAPL,ANET,2025-03-20 16:00:00-04:00,sell,10,82.97,2025-04-01 16:00:00-04:00,buy,10,78.49,12,4.48,0.053995,0.004392,1
1,AAPL,ANET,2025-03-21 16:00:00-04:00,sell,10,83.13,2025-04-01 16:00:00-04:00,buy,10,78.49,11,4.64,0.055816,0.00495,1
2,AAPL,ANET,2025-03-24 16:00:00-04:00,sell,10,87.51,2025-04-01 16:00:00-04:00,buy,10,78.49,8,9.02,0.103074,0.012338,1
3,AAPL,ANET,2025-03-25 16:00:00-04:00,sell,10,86.94,2025-04-01 16:00:00-04:00,buy,10,78.49,7,8.45,0.097193,0.013339,1
4,AAPL,ANET,2025-03-26 16:00:00-04:00,sell,10,81.66,2025-04-01 16:00:00-04:00,buy,10,78.49,6,3.17,0.038819,0.006368,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1965,XOM,SWKS,2025-03-25 16:00:00-04:00,buy,10,68.22,2025-03-27 16:00:00-04:00,sell,10,66.3,2,-1.92,-0.028144,-0.014172,0
1966,XOM,SWKS,2025-03-26 16:00:00-04:00,buy,10,67.6,2025-03-27 16:00:00-04:00,sell,10,66.3,1,-1.3,-0.019231,-0.019231,0
1967,XOM,SWKS,2025-03-27 16:00:00-04:00,sell,10,66.3,2025-03-28 16:00:00-04:00,buy,10,64.12,1,2.18,0.032881,0.032881,1
1968,XOM,SWKS,2025-03-28 16:00:00-04:00,buy,10,64.12,2025-04-01 16:00:00-04:00,sell,10,64.62,4,0.5,0.007798,0.001944,1


In [64]:
ticker_pair = "TSLA_CAG"
news_ticker = ticker_pair.split("_")[0]
ticker = ticker_pair.split("_")[1]
results.loc[(results["news_ticker"] == news_ticker) & (results["ticker"] == ticker), :]

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
1426,TSLA,CAG,2025-03-27 16:00:00-04:00,buy,10,26.46,2025-03-28 16:00:00-04:00,sell,10,26.55,1,0.09,0.003401,0.003401,1
1427,TSLA,CAG,2025-03-28 16:00:00-04:00,sell,10,26.55,2025-04-01 16:00:00-04:00,buy,10,26.6,4,-0.05,-0.001883,-0.000471,0
1428,TSLA,CAG,2025-03-31 16:00:00-04:00,sell,10,26.67,2025-04-01 16:00:00-04:00,buy,10,26.6,1,0.07,0.002625,0.002625,1


In [65]:
df = utils.load_csv(f"{pa_dir}/{ticker_pair}.csv")
df.loc[
    ~df["median_rating_excl"].isna(),
    [
        "date",
        "coint_corr_ticker",
        "close",
        "ticker",
        "median_rating_excl",
        "entry_signal",
        "exit_signal",
    ],
]

,date,coint_corr_ticker,close,ticker,median_rating_excl,entry_signal,exit_signal
0,2025-03-27,CAG,26.46,TSLA,4.0,buy,buy
1,2025-03-28,CAG,26.55,TSLA,1.0,sell,sell
2,2025-03-31,CAG,26.67,TSLA,1.0,sell,sell
3,2025-04-01,CAG,26.6,TSLA,1.0,sell,sell


In [134]:
entry_type = "long_only"
# entry_type = "short_only"
# entry_type = "long_or_short"

entry = strategy_utils.get_class_instance(
    "SentiEntry",
    "./src/strategy/entry/senti_entry.py",
    entry_type=entry_type,
)
exit = strategy_utils.get_class_instance(
    "SentiExit",
    "./src/strategy/exit/senti_exit.py",
    entry_type=entry_type,
)

df_signal = entry.gen_entry_signal(df)
df_signal = exit.gen_exit_signal(df_signal)
df_signal

,date,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal
0,2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait
1,2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait
2,2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,wait
3,2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,wait
4,2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait
5,2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,wait
6,2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,wait
7,2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait
8,2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait
9,2025-03-14,EA,138.15,139.23,137.87,138.71,3065500,NFLX,Friday,3.0,3,wait,wait


In [166]:
trades = strategy_utils.get_class_instance(
    "SentiTrades",
    "./src/strategy/trade/senti_trades.py",
    entry_struct="multiple",
    exit_struct="half_lifo",
    num_lots=10,
    percent_loss=0.01,
    exit_method="nearest_loss",
    monitor_close=False,
)

df_trades, df_pa = trades.gen_trades(df_signal)
df_trades

idx : 0
dt : 2025-03-03 16:00:00-05:00
close : 129.23
ent_sig : wait
ex_sig : wait
net_pos : 0
net_pos after update : 0
len(self.open_trades) : 0
open_trades : []

idx : 1
dt : 2025-03-04 16:00:00-05:00
close : 131.82
ent_sig : wait
ex_sig : wait
net_pos : 0
net_pos after update : 0
len(self.open_trades) : 0
open_trades : []

idx : 2
dt : 2025-03-05 16:00:00-05:00
close : 134.05
ent_sig : buy
ex_sig : wait
net_pos : 0
net_pos after update : 10
len(self.open_trades) : 1
open_trades : 
[
   {
      ticker: 'EA', ent_dt: '2025-03-05', ent_act: 'buy', ent_lots: 10, ent_price: 134.05, ex_dt: None, ex_act: None, ex_lots: 0, ex_price: None
   },
]

idx : 3
dt : 2025-03-06 16:00:00-05:00
close : 136.79
ent_sig : buy
ex_sig : wait
net_pos : 10
stop_price [long] : 132.71 -> monitor low [current: 133.37]
net_pos after update : 20
len(self.open_trades) : 2
open_trades : 
[
   {
      ticker: 'EA', ent_dt: '2025-03-05', ent_act: 'buy', ent_lots: 10, ent_price: 134.05, ex_dt: None, ex_act: None, ex_

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,NFLX,EA,2025-03-05 16:00:00-05:00,buy,10,134.05,2025-03-11 16:00:00-04:00,sell,10,139.03,5,4.98,0.037150,0.007322,1
1,NFLX,EA,2025-03-06 16:00:00-05:00,buy,10,136.79,2025-03-11 16:00:00-04:00,sell,10,139.03,4,2.24,0.016375,0.004069,1
2,NFLX,EA,2025-03-10 16:00:00-04:00,buy,10,140.43,2025-03-11 16:00:00-04:00,sell,10,139.03,1,-1.40,-0.009969,-0.009969,0
3,NFLX,EA,2025-03-11 16:00:00-04:00,buy,10,137.88,2025-03-12 16:00:00-04:00,sell,10,136.50,1,-1.38,-0.010009,-0.010009,0
4,NFLX,EA,2025-03-17 16:00:00-04:00,buy,10,142.9,2025-03-20 16:00:00-04:00,sell,10,141.47,3,-1.43,-0.010007,-0.003347,0
5,NFLX,EA,2025-03-18 16:00:00-04:00,buy,10,141.87,2025-03-20 16:00:00-04:00,sell,10,141.47,2,-0.40,-0.002819,-0.001410,0
6,NFLX,EA,2025-03-19 16:00:00-04:00,buy,10,141.97,2025-03-20 16:00:00-04:00,sell,10,141.47,1,-0.50,-0.003522,-0.003522,0
7,NFLX,EA,2025-03-20 16:00:00-04:00,buy,5,143.25,2025-03-21 16:00:00-04:00,sell,5,144.3,1,1.05,0.007330,0.007330,1
8,NFLX,EA,2025-03-25 16:00:00-04:00,buy,10,144.5,2025-03-27 16:00:00-04:00,sell,10,145.3,2,0.8,0.005536,0.002764,1
9,NFLX,EA,2025-03-24 16:00:00-04:00,buy,3,144.55,2025-03-27 16:00:00-04:00,sell,3,145.3,3,0.75,0.005189,0.001727,1


In [167]:
df_pa

,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal,stop_price,triggered
date,,,,,,,,,,,,,,
2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait,NaN,NaN
2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait,NaN,NaN
2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,wait,NaN,NaN
2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,wait,132.71,0
2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait,135.42,0
2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,wait,135.42,0
2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,wait,139.03,1
2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait,136.50,1
2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait,NaN,NaN


In [107]:
pos = "long"
percent_loss = 0.01
trades = [
    # (10, 134.05),
    # (5, 136.79),
    # (3, 140.43),
    (10, 142.9),
    (10, 141.87),
    (10, 141.97),
    # (2, 137.88),
    # (10, 144.5),
]
total_lots = sum(trade[0] for trade in trades)
total_invest = sum(trade[0] * trade[1] for trade in trades)

# stop_price = (
#     total_invest * (1 - percent_loss) / total_lots
#     if pos == "long"
#     else total_invest * (1 + percent_loss) / total_lots
# )
# stop_price

stop_list = [
    trade[1] * (1 - percent_loss) if pos == "long" else trade[1] * (1 + percent_loss)
    for trade in trades
]
stop_list

[141.471, 140.4513, 140.5503]

In [103]:
134.05 * (1 - percent_loss)

132.70950000000002